In [ ]:
!pip install -U datasets

In [ ]:
# %% 1. การติดตั้งและ Import Library

# ติดตั้ง Library ที่จำเป็น
!pip install -q transformers[torch] torch accelerate scikit-learn

import pandas as pd
import numpy as np
import torch
import json
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.auto import tqdm

# ตรวจสอบและตั้งค่าอุปกรณ์ (GPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"อุปกรณ์ที่ใช้: {DEVICE}")


In [ ]:
# %% 2. การโหลดและเตรียมข้อมูล (CORD Dataset)

print("กำลังโหลดและประมวลผลข้อมูล CORD...")
# โหลดข้อมูล CORD จาก Hugging Face Hub (ใช้เฉพาะส่วน train มาสาธิต)
raw_dataset = load_dataset("naver-clova-ix/cord-v2", split="train")

def create_example_from_cord(example):
    """
    ฟังก์ชันสำหรับแปลง 1 ตัวอย่างข้อมูลจาก CORD ให้มี invoice_text และ ground_truth_json
    """
    ground_truth = json.loads(example["ground_truth"])
    
    # สร้าง invoice_text โดยการต่อข้อความจากทุกบรรทัด
    full_text = " ".join([line['text'] for line in ground_truth['valid_line']])
    
    # สกัดข้อมูลเป้าหมาย
    company_name = ground_truth['meta']['store_name']
    invoice_date = ground_truth['meta']['payment_date']
    total_amount = ground_truth['meta']['total_price']
    
    # สร้าง JSON ground truth สำหรับ few-shot example
    ground_truth_dict = {
        "company_name": company_name,
        "invoice_date": invoice_date,
        "total_amount": total_amount
    }
    
    return {
        "invoice_text": full_text,
        "ground_truth_json": json.dumps(ground_truth_dict, ensure_ascii=False)
    }

# ประมวลผลและสร้าง DataFrame
processed_dataset = raw_dataset.map(create_example_from_cord, remove_columns=raw_dataset.column_names)
df = processed_dataset.to_pandas().dropna().sample(frac=1, random_state=42).reset_index(drop=True)


# --- คัดเลือกตัวอย่างคุณภาพสูงสำหรับ Few-Shot Prompt ---
few_shot_example_1 = df.iloc[0]
few_shot_example_2 = df.iloc[1]

# --- สร้างข้อมูลสำหรับ Test Set เพื่อจำลองการแข่งขัน ---
test_df = df.iloc[10:110].copy() # ลดขนาดเพื่อความรวดเร็วในการทดลอง
test_df['invoice_id'] = [f"inv_{i:03d}" for i in range(len(test_df))]

print("คัดเลือกตัวอย่าง Few-shot และเตรียม Test set สำเร็จ")
print("\n--- ตัวอย่าง Few-shot 1 ---")
print(f"Text: {few_shot_example_1['invoice_text'][:100]}...")
print(f"JSON: {few_shot_example_1['ground_truth_json']}")


In [ ]:
# %% 3. การโหลดโมเดล LLM และ Tokenizer

model_id = "Qwen/Qwen2-7B-Instruct"
print(f"กำลังโหลด Tokenizer และโมเดล LLM: '{model_id}'...")

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, # ใช้ bfloat16 เพื่อลดการใช้ VRAM
    device_map="auto"           # โหลดโมเดลข้าม GPU อัตโนมัติ
)
model.eval() # ตั้งเป็น evaluation mode

print("โหลดโมเดล LLM สำเร็จ")


In [ ]:
# %% 4. การออกแบบ Prompt (Few-Shot & JSON Output)

def create_extraction_prompt(invoice_text):
    """
    สร้าง Prompt ที่สมบูรณ์แบบ Few-shot และสั่งให้ตอบเป็น JSON
    """
    chat_template = [
        # 1. System Message: กำหนดบทบาท, คำสั่งหลัก, และกฎ
        {
            "role": "system",
            "content": """You are an expert data extraction assistant. Your task is to extract specific information from an invoice text.
You must respond ONLY with a valid JSON object containing the following keys: 'company_name', 'invoice_date', 'total_amount'.
If a piece of information cannot be found in the text, you must use the value null."""
        },

        # 2. Few-shot Example 1
        {
            "role": "user",
            "content": f"Extract information from the following invoice text:\n\n---\n{few_shot_example_1['invoice_text']}\n---"
        },
        {
            "role": "assistant",
            "content": few_shot_example_1['ground_truth_json']
        },
        
        # 3. Few-shot Example 2
        {
            "role": "user",
            "content": f"Extract information from the following invoice text:\n\n---\n{few_shot_example_2['invoice_text']}\n---"
        },
        {
            "role": "assistant",
            "content": few_shot_example_2['ground_truth_json']
        },

        # 4. User Query (ข่าวที่เราต้องการวิเคราะห์)
        {
            "role": "user",
            "content": f"Extract information from the following invoice text:\n\n---\n{invoice_text}\n---"
        }
    ]
    return chat_template

# ทดสอบสร้าง Prompt
test_prompt = create_extraction_prompt("This is a test invoice.")
print("สร้าง Template Prompt สำเร็จ")
# print(json.dumps(test_prompt, indent=2)) # สามารถ uncomment เพื่อดูโครงสร้างเต็มๆ ได้


In [ ]:
# %% 5. การทำนายผลด้วย LLM และการ Parse JSON

results = []
default_json = {"company_name": None, "invoice_date": None, "total_amount": None}

for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Extracting with LLM"):
    # สร้าง Prompt สำหรับ invoice แต่ละใบ
    prompt = create_extraction_prompt(row['invoice_text'])
    
    # แปลง Prompt เป็น Token IDs และส่งเข้า GPU
    inputs = tokenizer.apply_chat_template(
        prompt, tokenize=True, add_generation_prompt=True, return_tensors="pt"
    ).to(DEVICE)

    # จำกัดความยาว Input เพื่อป้องกัน Error
    if inputs.shape[1] > 4096:
        inputs = inputs[:, -4096:]

    # สั่งให้โมเดลสร้างข้อความ
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=128,         # จำกัดความยาวของคำตอบให้พอดีกับ JSON
            do_sample=False,            # ไม่ต้องสุ่มคำตอบเพื่อให้ผลลัพธ์คงที่
            pad_token_id=tokenizer.eos_token_id
        )
    
    # ถอดรหัสคำตอบ
    response_text = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)

    # พยายาม Parse JSON จากคำตอบอย่างรัดกุม
    try:
        # ค้นหา JSON object แรกที่เจอใน response
        start_index = response_text.find('{')
        end_index = response_text.rfind('}') + 1
        json_part = response_text[start_index:end_index]
        parsed_json = json.loads(json_part)
        # ตรวจสอบว่า key ที่จำเป็นอยู่ครบหรือไม่
        for key in default_json.keys():
            if key not in parsed_json:
                parsed_json[key] = None
        results.append(parsed_json)
    except (json.JSONDecodeError, AttributeError, ValueError):
        results.append(default_json.copy()) # หากเกิดข้อผิดพลาดในการ parse ให้ใช้ค่า default

print("\nสกัดข้อมูลด้วย LLM สำเร็จ")


In [ ]:
# %% 6. การสร้างไฟล์ Submission

# สร้าง DataFrame จาก list ของผลลัพธ์
submission_df = pd.DataFrame(results)

# ตรวจสอบและจัดเรียงคอลัมน์ให้ตรงตามโจทย์
submission_df = submission_df[['company_name', 'invoice_date', 'total_amount']]

# แทรก invoice_id จาก test_df เข้าไปเป็นคอลัมน์แรก
submission_df.insert(0, 'invoice_id', test_df['invoice_id'].values)

print("\nตัวอย่างข้อมูลในไฟล์ Submission:")
print(submission_df.head())

# บันทึกเป็นไฟล์ CSV
submission_df.to_csv("submission_llm_extraction.csv", index=False)

print("\nสร้างไฟล์ submission_llm_extraction.csv สำเร็จ!")

# (ทางเลือก) ดูผลลัพธ์เทียบกับ Ground Truth
print("\n--- เปรียบเทียบผลลัพธ์กับ Ground Truth (เพื่อการประเมิน) ---")
comparison_df = test_df[['invoice_id']].copy()
comparison_df['ground_truth'] = test_df['ground_truth_json'].apply(json.loads)
comparison_df['prediction'] = results
print(comparison_df.head())
